# Smart ou Ultra?
* A operadora de celular Megaline está insatisfeita com o fato de muitos de seus clientes estarem usando planos antigos. Eles querem desenvolver um modelo que possa analisar o comportamento do cliente e recomendar um dos planos mais recentes da Megaline: Smart ou Ultra.
* Você tem acesso a dados de comportamento dos assinantes que já mudaram para os novos planos (do projeto do curso de Análise de Dados Estatísticos). Para esta tarefa de classificação, você precisa desenvolver um modelo que escolherá o plano certo. Como você já executou a etapa de pré-processamento de dados, pode ir direto para a criação do modelo.
* Desenvolva um modelo com a maior acurácia possível. Neste projeto, o limite para acurácia é 0,75. Verifique a acurácia usando o conjunto de dados de teste.


## Iniciação & Vizualização
* Carregar as bibliotecas e vizualiazção de dados

In [31]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import joblib
from joblib import dump

In [2]:
df = pd.read_csv('/datasets/users_behavior_upd.csv')
df

,calls,minutes,messages,mb_used,is_ultimate
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


## Criação da Feature e Target

In [3]:
features = df.drop(['is_ultimate'], axis=1)
target = df['is_ultimate']
print(features.shape)
print(target.shape)

(3214, 4)
(3214,)


## Criação dos modelos de Treinamento

In [4]:
model = DecisionTreeClassifier(random_state=12345)

model.fit(features, target)

DecisionTreeClassifier(random_state=12345)

## Divisão dos Dados em Treinamento, Teste e Validação

In [30]:
df_train, df_meio = train_test_split(df, test_size=0.4, random_state=12345)

df_valid, df_test = train_test_split(df_meio, test_size=0.5, random_state=12345)

features_train = df_train.drop(['is_ultimate'], axis=1)
target_train = df_train['is_ultimate']

features_test = df_test.drop(['is_ultimate'], axis=1)
target_test = df_test['is_ultimate'] 

features_valid = df_valid.drop(['is_ultimate'], axis=1)
target_valid = df_valid['is_ultimate'] 


print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


* Todos os conjuntos de dados com seus respectivos valores (treinamento 60%, validação 20%, teste 20%)

## Teste DecisionTreeClassifier



In [6]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train,target_train) 
    predictions = model.predict(features_train) 
    result = accuracy_score(target_train, predictions) 
    if result > best_result:
        best_model = model
        best_result = result
        
print("Acurácia do melhor modelo:", best_result)
print(depth)

Acurácia do melhor modelo: 0.8890041493775933
10


In [7]:
best_model = None
best_result = 0
for depth in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_test,target_test) 
    predictions = model.predict(features_test) 
    result = accuracy_score(target_test, predictions) 
    if result > best_result:
        best_model = model
        best_result = result
        
print("Acurácia do melhor modelo:", best_result)
print(depth)

Acurácia do melhor modelo: 0.9097978227060654
10


* Conjunto de treinamento e de teste tem resultados semelhantes

In [15]:
for depth in range(1, 11):
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
        model.fit(features_test,target_test)
        
        
        predictions_valid = model.predict(features_valid)

        print("max_depth =", depth, ": ", end='')
        print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7511664074650077
max_depth = 2 : 0.7869362363919129
max_depth = 3 : 0.7838258164852255
max_depth = 4 : 0.7869362363919129
max_depth = 5 : 0.76049766718507
max_depth = 6 : 0.7698289269051322
max_depth = 7 : 0.7636080870917574
max_depth = 8 : 0.7682737169517885
max_depth = 9 : 0.7620528771384136
max_depth = 10 : 0.7589424572317263


* O melhor modelo de DecisionTreeClassifier para o conjunto de validação é com max_depth= 2 ou 4 com a precisão de **78.69%**

### Vizualização do Melhor Hiperparâmetro

In [16]:
finalmodelt= DecisionTreeClassifier(random_state=12345, max_depth=4)
finalmodelt.fit(features_test,target_test)

DecisionTreeClassifier(max_depth=4, random_state=12345)

## Teste RandomForestClassifier

In [34]:
best_score = 0
best_est = 0
for est in range(1, 21): 
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train,target_train) 
    score = model.score(features_valid,target_valid) 
    if score > best_score:
        best_score = score
        best_est = est

print("A acurácia do melhor modelo no conjunto de validação (n_estimators = {}): {}".format(best_est, best_score))

A acurácia do melhor modelo no conjunto de validação (n_estimators = 18): 0.7931570762052877


* O melhor modelo de RandomForestClassifier para o conjunto de validação é com n_estimators=18 com a precisão de **79.31%**

### Vizualização do Melhor Hiperparâmetro

In [35]:
final_modelrt = RandomForestClassifier(random_state=54321, n_estimators=18) 
final_modelrt.fit(features_train, target_train)

RandomForestClassifier(n_estimators=18, random_state=54321)

## Teste LogisticRegression

In [25]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train) 
score_train = model.score(features_train,target_train) 
score_valid = model.score(features_valid, target_valid) 

print("Acurácia do modelo de regressão logística no conjunto de treinamento:", score_train)
print("Acurácia do modelo de regressão logística no conjunto de validação:", score_valid)

Acurácia do modelo de regressão logística no conjunto de treinamento: 0.7510373443983402
Acurácia do modelo de regressão logística no conjunto de validação: 0.7573872472783826


* O modelo de LogisticRegression teve uma precisão de **75.7%** no conjunto de validação

## Exemplo de como salvar modelos


* Usando joblib vou salvar o melhor modelo de DecisionTreeClassifier para o exemplo

In [27]:
dump(finalmodelt, 'finalmodelt.joblib') 

['finalmodelt.joblib']

In [28]:
modelo = joblib.load('finalmodelt.joblib')
modelo

DecisionTreeClassifier(max_depth=4, random_state=12345)

## Conclusões

### Conclusão DecisionTreeClassifier

* Este modelo teve uma boa precisão com **78.69%**, porém não foi a maior, mas nesse caso teve um bom desempenho. Normalmente este modelo tem baixa precisão e alta velocidade, confirmou-se a velocidade.

### Conclusão RandomForestClassifier

* Este modelo teve a maior precisão com **79.31%**. Normalmente este modelo tem alta precisão com uma baixa velocidade, o que se confirmou neste projeto. 

### Conclusão LogisticRegression

* Este modelo teve a mais baixa precisão com apenas **75.73%**. Normalmente este modelo tem uma precisão média, o que não se comprovou neste projeto, porém sua velocidade de fato é alta.

## Conclusão Geral

* Acredito que o modelo que mais tenha se destacado foi o RandomForestClassifier por ter confirmado as espectativas, com a maior precisão de quase 80%, sua velocidade de fato foi a mais baixa mas acredito que isso não seja um impecilio pois sua precisão alta faz valer a pena. Escolho este modelo também pelo método que trabalha, acredito ser o ideal para esta situação.